In [1]:
from wsp.camera.camera import BaseCamera
from wsp.utils.paths import WSP_PATH, CONFIG_PATH
from wsp.utils.utils import loadconfig
import time


class WinterDeepCamera(BaseCamera):
    
    """
    Winter Deep Camera implementation.
    This class extends BaseCamera to provide specific functionality for the Winter Deep camera.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Additional initialization if needed

config = loadconfig(CONFIG_PATH)

cam = WinterDeepCamera(base_directory = WSP_PATH,
                    config = config,
                    camname = "WINTER-deep",
                    daemon_pyro_name = "FakeCamera",
                    logger = None,
                    verbose=False)



telescope: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp


In [2]:
cam.print_state()

state = cam.state
print("Camera state:", state["camera_state"])

{
  "timestamp": 1756256448.947628,
  "camname": "WINTER-deep",
  "is_connected": true,
  "camera_state": "OFF",
  "imdir": "",
  "imname": "",
  "imstarttime": "",
  "imtype": null,
  "immode": null,
  "tec_temp": 0,
  "tec_enabled": 0,
  "tec_setpoint": 0.0,
  "tec_voltage": 0.0,
  "tec_current": 0.0,
  "tec_percentage": 0.0,
  "exptime": 0.0
}
Camera state: OFF


In [3]:
print("Starting up camera...")
sendtime = time.time()
cam.startupCamera()

while cam.state["camera_state"] != "READY":

    #print("Waiting for camera to be ready...")
    cam.update_state()
endtime = time.time()

print(f"Camera startup took {endtime - sendtime:.1f} seconds")
print("Camera is READY.")


Starting up camera...
WINTER-deep local interface: State changed: 2025-08-26 21:00:48.951393: OFF -> STARTUP_REQUESTED
WINTER-deep local interface: State changed: 2025-08-26 21:00:51.140081: STARTUP_REQUESTED -> READY
Camera startup took 2.2 seconds
Camera is READY.


In [4]:
cam.print_state()

{
  "timestamp": 1756256451.144326,
  "camname": "WINTER-deep",
  "is_connected": true,
  "camera_state": "READY",
  "imdir": "",
  "imname": "",
  "imstarttime": "",
  "imtype": null,
  "immode": null,
  "tec_temp": 0,
  "tec_enabled": 0,
  "tec_setpoint": 0.0,
  "tec_voltage": 0.0,
  "tec_current": 0.0,
  "tec_percentage": 0.0,
  "exptime": 0.0
}


In [5]:
cam.setExposure(10)
cam.print_state()

WINTER-deep local interface: State changed: 2025-08-26 21:00:51.148217: READY -> SETTING_PARAMETERS
WINTER-deep local interface: State changed: 2025-08-26 21:00:51.148620: SETTING_PARAMETERS -> READY
{
  "timestamp": 1756256451.149005,
  "camname": "WINTER-deep",
  "is_connected": true,
  "camera_state": "READY",
  "imdir": "",
  "imname": "",
  "imstarttime": "",
  "imtype": null,
  "immode": null,
  "tec_temp": 0,
  "tec_enabled": 0,
  "tec_setpoint": 0.0,
  "tec_voltage": 0.0,
  "tec_current": 0.0,
  "tec_percentage": 0.0,
  "exptime": 0.0
}


In [6]:
print("Going to do an exposure:")
print(f"Camera State: {cam.state['camera_state']}")


print("Doing exposure...")
print(f"time = {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")
sendtime = time.time()

cam.doExposure()
#time.sleep(1.0)  # Wait for daemon to process
# first wait until it registers that camera_state is EXPOSING
print("Waiting for exposure to complete...")
print(f"Camera State: {cam.state['camera_state']}")

while cam.state["camera_state"] in ("EXPOSING", "SETTING_PARAMETERS"):
    cam.update_state()
    time.sleep(0.1)
    
endtime = time.time()
print(f"Camera exposure took {endtime - sendtime:.1f} seconds")
print(f"Camera State: {cam.state['camera_state']}")


Going to do an exposure:
Camera State: READY
Doing exposure...
time = 2025-08-26 17:00:51
WINTER-deep local interface: running doExposure
WINTER-deep local interface: State changed: 2025-08-26 21:00:51.153329: READY -> EXPOSING
WINTER-deep local interface: updating state dictionaries
WINTER-deep local interface: making FITS header
WINTER-deep local interface: sending doExposure request to camera: imdir = /Users/nlourie/data/images/20250826, imname = FakeCamera_20250826-210051-153
Waiting for exposure to complete...
Camera State: EXPOSING
WINTER-deep local interface: State changed: 2025-08-26 21:01:02.202103: EXPOSING -> READY
Camera exposure took 11.2 seconds
Camera State: READY
